In [3]:
import pandas as pd
import numpy as np
import json
import math

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [7]:
a = ["horario","atrito"]
b = [
    {"id_assunto":170,"horario":"sim","atrito":"sim"},
    {"id_assunto":160,"horario":"sim","atrito":"não"},
    {"id_assunto":170,"horario":"não","atrito":"não"}
]
c = ["id_assunto","horario"]

In [90]:
teste = pd.DataFrame(b)

In [91]:
teste

,id_assunto,horario,atrito
0,170,sim,sim
1,160,sim,não
2,170,não,não


In [92]:
features = teste[c]

In [93]:
features

,id_assunto,horario
0,170,sim
1,160,sim
2,170,não


In [5]:
with open("Datassets/myjsonfile.json",'r',encoding='UTF-8') as file:
    datasset = json.load(file)

In [11]:
class Modeling:
    def __init__(self,datasset,targets,drops=None) -> None: #features
        
        dataframe = pd.DataFrame(datasset)
        self.dataframe = dataframe.dropna()

        if drops != None:
            for i in drops:
                self.dataframe = self.dataframe.drop(i,axis=1)


        self.target = self.dataframe[targets]
        self.features = self.dataframe.drop(targets,axis=1)

        self.uniqueTargetsCount= {i:{"UniqueValues":np.bincount(self.dataframe[i]),"UniqueProbability":np.bincount(self.dataframe[i])/(len(self.dataframe[i])),"informationGain":{}} for i in targets}
    
        for k in self.uniqueTargetsCount:

            for j in self.features:

                self.uniqueTargetsCount[k]["informationGain"][j] = self.information_gain(self.dataframe,j,k,self.uniqueTargetsCount[k]["UniqueProbability"])

        self.encoded_features = self.label_encoder(self.features)
        self.X_train, self.X_test, self.y_train, self.y_test = self.train_test(self.encoded_features,self.target)
    
    def calc_entropy(self,column,UniqueProbability):
        # Conta o número de ocorrências de cada valor único na coluna
        #UniqueProbability
        
        # Calcula a probabilidade de cada valor dividindo pela quantidade total de elementos na coluna
        probabilities = UniqueProbability / len(column)
        
        entropy = 0  # Valor inicial da entropia
        
        # Loop para calcular a entropia de cada valor único
        for prob in probabilities:
            if prob > 0:
                entropy += prob * math.log(prob, 2)  # Calcula a entropia de cada valor e soma ao total

        return -entropy  # Retorna o negativo da entropia conforme a fórmula
    
    def information_gain(self,data, split,target,UniqueProbability):

        original_entropy=self.calc_entropy(data[target],UniqueProbability)

        values=data[split].unique()

        if len(values) < 2:
            # Se houver menos de dois valores únicos, não é possível dividir, então o ganho de informação é 0
            return 0

        left_split=data[data[split]==values[0]]

        right_split=data[data[split]==values[1]]

        subract=0

        for subset in [left_split,right_split]:
            
            prob=(subset.shape[0])/data.shape[0]
            subract += prob * self.calc_entropy(subset[target],UniqueProbability)

        return  original_entropy - subract
    
    def label_encoder(self,X):
        Encoder_X=LabelEncoder()

        for col in X.columns:
            X[col]=Encoder_X.fit_transform(X[col])
        
        return X
    
    def train_test(self,X,y):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        return X_train, X_test, y_train, y_test


In [9]:
myModeling = Modeling(datasset['data'],['id_assunto'])

In [10]:
myModeling.X_test.describe()

,mensagem_resposta,data_hora_analise,data_hora_encaminhado,data_hora_assumido,data_hora_execucao,id_contrato_kit,preview,data_agenda_final,id,tipo,...,data_prazo_limite,data_reservada,id_ticket,origem_endereco,justificativa_sla_atrasado,origem_endereco_estrutura,data_reagendar,data_prev_final,origem_cadastro,ultima_atualizacao
count,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.0,8800.000000,8800.000000,8800.000000,...,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000
mean,8946.474432,35.259773,4013.182273,0.188295,2023.792159,7124.663750,0.0,78.441932,22006.164773,1.105795,...,30.184091,39.130909,7231.446364,2.423182,4.754091,1.186932,49.792386,0.464659,0.970455,19890.866705
std,7124.408386,192.202868,5511.009231,0.429479,3752.606331,5023.418019,0.0,353.220574,12691.874633,0.308700,...,106.799258,57.312792,6483.658337,1.034910,42.513489,0.452732,259.399589,0.498778,0.169339,11621.978512
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,6.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,1738.000000,0.000000,0.000000,0.000000,0.000000,2671.750000,0.0,0.000000,10990.750000,1.000000,...,0.000000,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,9785.000000
50%,7058.000000,0.000000,1.000000,0.000000,0.000000,7057.000000,0.0,1.000000,22161.000000,1.000000,...,1.000000,1.000000,6307.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,19975.000000
75%,15689.000000,0.000000,8054.250000,0.000000,2236.500000,11419.250000,0.0,1.000000,32896.250000,1.000000,...,1.000000,80.000000,12895.500000,3.000000,0.000000,1.000000,1.000000,1.000000,1.000000,29936.500000
max,21800.000000,1592.000000,17318.000000,12.000000,13463.000000,16209.000000,0.0,2442.000000,43999.000000,2.000000,...,657.000000,192.000000,19554.000000,4.000000,570.000000,2.000000,2131.000000,1.000000,1.000000,40149.000000
